In [1]:
import numpy as np
from latticegas import LatticeGas as latgas

In [2]:
variant = 7
N = 16 

In [3]:
n_x = 420
n_y = 180

xc = int(n_x/4)
yc = int(n_y/2)
r = int(n_x/9)

xc, yc, r

(105, 90, 46)

In [4]:
u_lb_min = 0.01
u_lb_max = 0.1
Re_min = 20
Re_max = 1000

In [5]:
u_lb = u_lb_min + (variant - 1)*(u_lb_max - u_lb_min)/(N-1)
Re = int(Re_min + (variant - 1)*(Re_max - Re_min)/(N-1))

u_lb, Re

(0.046000000000000006, 412)

In [6]:
parametrs = {'nx':n_x, 'ny':n_y, 'u_lb':u_lb, 'Re':Re}
obstacle = {'xc':xc, 'yc':yc, 'r':r}

In [11]:
model = latgas(parametrs, obstacle)

In [12]:
%%time
model.solve(n_step=1000)

E:\PROGRAMMING PROJECT\Python project\BMSTU\6th year\LatticeGas\latticegas.py:156: RuntimeWarning: overflow encountered in multiply
  self.f_out[i,:,:] = self.f_in[i,:,:] - self.omega*(self.f_in[i,:,:]\
E:\PROGRAMMING PROJECT\Python project\BMSTU\6th year\LatticeGas\latticegas.py:141: RuntimeWarning: overflow encountered in multiply
  return density*self._a[i]*(1 + 3*u@self._v[i] + 4.5*(u@self._v[i])**2 - 1.5*np.linalg.norm(u, axis=-1)**2)
C:\Users\Maksim\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
E:\PROGRAMMING PROJECT\Python project\BMSTU\6th year\LatticeGas\latticegas.py:141: RuntimeWarning: invalid value encountered in add
  return density*self._a[i]*(1 + 3*u@self._v[i] + 4.5*(u@self._v[i])**2 - 1.5*np.linalg.norm(u, axis=-1)**2)
E:\PROGRAMMING PROJECT\Python project\BMSTU\6th year\LatticeGas\latticegas.py:141: RuntimeWarning: invalid value encountered in matmul


CPU times: total: 1min 19s
Wall time: 1min 20s


In [13]:
density = model.field_den
velosity = model.field_u
pressure = model.field_p

In [ ]:
np.